# 신경망(NEURAL NETWORKS)
- 신경망은 torch.nn 패키지를 사용하여 생성
- nn 은 모델을 정의하고 미분하는데 autograd 를 사용
- nn.Module 은 계층(layer)과 output 을 반환하는 forward(input) 메서드를 포함

## 신경망 정의하기

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)   # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 크기가 정사각형인 경우 단일 숫자만 지정할 수 있습니다.
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())    # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1231, -0.0109,  0.1228,  0.0861,  0.0165, -0.0061,  0.0329, -0.0552,
          0.0275, -0.1169]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1,10))

#### torch.nn 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니-배치만을 입력
#### nnConv2D 는 nSamples x nChannels x Height x Width 의 4차원 Tensor를 입력

## 손실함수(Loss Function)

In [6]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)    # view 차원 맞추기
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7135, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## 역전파(Backprop)
- 오차를 역전파하기 위해서는 loss.backward()만 해주면 됨.
- loss.bacward()를 호출하여 역전파 전과 후에 conv1의 bias gradient를 확인

In [8]:
net.zero_grad()   # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0013, -0.0104, -0.0040,  0.0054,  0.0106, -0.0097])


## 가중치 갱신
- 확률적 경사하강법

In [9]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [10]:
import torch.optim as optim

# Optimizer 생성
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과저에서는 다음과 같다.
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()